## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.60,-73.97,81.0,67,32,13.40,scattered clouds
1,1,Thompson,CA,55.74,-97.86,75.2,38,20,4.70,few clouds
2,2,Cayenne,GF,4.93,-52.33,84.2,70,67,5.82,light rain
3,3,Boueni,YT,-12.90,45.08,75.2,78,0,8.05,clear sky
4,4,Ushuaia,AR,-54.80,-68.30,41.0,75,40,24.16,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?70
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)&
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.60,-73.97,81.00,67,32,13.40,scattered clouds
1,1,Thompson,CA,55.74,-97.86,75.20,38,20,4.70,few clouds
2,2,Cayenne,GF,4.93,-52.33,84.20,70,67,5.82,light rain
3,3,Boueni,YT,-12.90,45.08,75.20,78,0,8.05,clear sky
7,7,Vaini,TO,-21.20,-175.20,73.40,88,75,8.05,broken clouds
...,...,...,...,...,...,...,...,...,...,...
673,673,Yinchuan,CN,38.47,106.27,80.33,16,100,6.73,overcast clouds
675,675,Grand-Santi,GF,4.25,-54.38,82.09,91,58,2.30,moderate rain
676,676,Obera,AR,-27.49,-55.12,79.59,45,0,2.71,clear sky
677,677,Concepcion De Buenos Aires,MX,19.97,-103.27,75.20,53,40,4.70,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Albany,US,42.60,-73.97,81.0,67,32,13.40,scattered clouds
1,1,Thompson,CA,55.74,-97.86,75.2,38,20,4.70,few clouds
2,2,Cayenne,GF,4.93,-52.33,84.2,70,67,5.82,light rain
3,3,Boueni,YT,-12.90,45.08,75.2,78,0,8.05,clear sky
7,7,Vaini,TO,-21.20,-175.20,73.4,88,75,8.05,broken clouds


In [7]:
import numpy as np

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = np.nan
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,81.00,scattered clouds,42.60,-73.97,NaN
1,Thompson,CA,75.20,few clouds,55.74,-97.86,NaN
2,Cayenne,GF,84.20,light rain,4.93,-52.33,NaN
3,Boueni,YT,75.20,clear sky,-12.90,45.08,NaN
7,Vaini,TO,73.40,broken clouds,-21.20,-175.20,NaN
10,Atikokan,CA,70.00,broken clouds,48.76,-91.62,NaN
13,Hilo,US,77.00,overcast clouds,19.73,-155.09,NaN
14,Samarai,PG,76.84,light rain,-10.62,150.67,NaN
18,Bambanglipuro,ID,70.38,broken clouds,-7.95,110.28,NaN
21,Dharur,IN,76.35,moderate rain,18.82,76.12,NaN


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Albany,US,81.0,scattered clouds,42.60,-73.97,NaN
1,Thompson,CA,75.2,few clouds,55.74,-97.86,Burntwood Hotel
2,Cayenne,GF,84.2,light rain,4.93,-52.33,Hôtel Le Dronmi
3,Boueni,YT,75.2,clear sky,-12.90,45.08,Auberge bouloulou( inter djasmire )
7,Vaini,TO,73.4,broken clouds,-21.20,-175.20,Keleti Beach Resort


In [11]:
hotel_df.count()

City                   334
Country                334
Max Temp               334
Current Description    334
Lat                    334
Lng                    334
Hotel Name             308
dtype: int64

In [12]:
# 7. Drop the rows where there is no Hotel Name
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Thompson,CA,75.2,few clouds,55.74,-97.86,Burntwood Hotel
2,Cayenne,GF,84.2,light rain,4.93,-52.33,Hôtel Le Dronmi
3,Boueni,YT,75.2,clear sky,-12.90,45.08,Auberge bouloulou( inter djasmire )
7,Vaini,TO,73.4,broken clouds,-21.20,-175.20,Keleti Beach Resort
10,Atikokan,CA,70.0,broken clouds,48.76,-91.62,Camp Quetico


In [26]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [27]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dd><dd>{Current Description}</dd>
</d1>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig
# 11b. Display the figure
display(fig)

Figure(layout=FigureLayout(height='420px'))